In [1]:
"""
- 심층 신경망

-2개의 층
케라스 API를 사용해서 패션 MNIST 데이터셋 불러오기
"""
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
"""
이미지의 픽셀값을 0~255 범위에서 0~1 사이로 변환하고
28X28 크기의 2차원 배열을 784 크기의 1차원 배열로 바꿈
사이킷런의 train_test_split()함수로 훈련세트와 검증세트 나눔
"""
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
train_scaled, train_target, test_size=0.2, random_state=42)

In [3]:
"""
인공 신경망 모델에 층을 2개 추가함

이렇게 입력층과 출력층 사이에 있는 모든 층을 은닉층 이라고 함

활성화 함수 - 신경망 층의 선형 방정식의 계산 값에 적용하는 함수

출력층에 적용했던 소프트맥스 함수도 활성화 함수입니다 
출력층에 적용하는 활성화 함수는 종류가 제한 되어 있다
이진 분류일 경우 시그모이드 함수, 다중 분류일 경우 소프트맥스 함수
은닉층의 활성화 함수는 비교적 자유롭습니다
대표적으로 시그모이드 함수와 렐루 함수 

책 읽어 보자 - 선형 계산을 적당하게 비선형적으로 비틀어 주어야합니다

모든 신경망의 은닉층에는 항상 활성화 함수가 있다

시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 
케라스의 Dense 클래스로 만들어 봄
케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해야함

---
dense1 은닉층이고 100개의 뉴런을 가진 밀지브
활성화 함수는 sigmoid로 지정  input_shape 매개변수에서 입력의 크기 (784)로 지정
은닉층의 뉴런 개수를 정하는 데는 특별한 기준이 없다 
- 몇 개의 뉴런을 두어야 할지 판단하기 위해서는 상당한 경험이 필요함

"""
dense1 = keras.layers.Dense(100, activation="sigmoid", input_shape=(784,))
dense2 = keras.layers.Dense(10, activation="softmax")

In [4]:
"""
-심층 신경망 만들기

dense1과 dense2 객체를 Sequential 클래스에 추가하여 심층 신경망 만들기
"""
model = keras.Sequential([dense1, dense2])

In [5]:
"""
Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 이와 같이 dense1과 dense2를 리스트로 만들어 전달
출력층을 가장 마지막에 두어야함
인공 신경망의 강력한 성능은 바로 이렇게 층을 추가하여 입력 데이터에 대해 
연속적인 학습을 진행하는 능력

케라스 모델의 summary()메서드 호출

출력의 크기를 보면 (None,100)임 
첫 번째 차원은 샘플의 개수를 나타냄 - 샘플 개수가 아직 정의되어 있지 않기 때문에 None
왜그럴까? - 케라스 모델의 fit()메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고
잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행함, 바로 미니배치 경사 하강법을 사용

케라스의 기본 미니배치 크기는 32개입니다. 이 값은 fit()메서드에서 batch_size 매개변수로 바꿀 수 있다
따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정
이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부름

두 번째 100, 은닉층의 뉴런 개수를 100개로 두었으니까 100개의 출력이 나옴
즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축

마지막에 모델 파라미터 개수가 출력됨
이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다
첫층은 784개 X 100개 + 100개 = 78500개
두 번째층은 100개 X 10개 + 10개 = 1010개
총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79510개 

은닉층ㅇ과 출력층의 파라미터 개수를 합친 값

그 아래 훈련되지 않는 파라미터(Non-trainable params)는 0 으로 나옴
"""
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [7]:
"""
- 층을 추가하는 다른방법
모델을 훈련하기 전에 Seqeuntial 클래스에 층을 추가하는 다른 방법

Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다
"""
model = keras.Sequential([
    keras.layers.Dense(100, activation="sigmoid", input_shape=(784,),
                      name="hidden"),
    keras.layers.Dense(10, activation="softmax", name="output")
], name="패션 MNIST 모델")

In [8]:
"""
Sequential 클래스의 name 매개변수로 모델의 이름을 지정
Dense층의 name 매개변수에 층의 이름을 hidden과 output으로 각각 지정

모델의 이름과 달리 층의 이름은 반드시 영문이어야함
summary()메서드 출력
"""
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
"""
이 방법이 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어짐
또 조건에 따라 층을 추가할 수도 없다
Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법은 모델의 add()메서드

이 방법은 Sequential 클래스의 객체를 만들고 이 객체의 add()메서드를 호출하여 층을 추가
"""
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(keras.layers.Dense(10, activation="softmax"))

In [10]:
"""
summary()메서드
"""
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [11]:
"""
모델을 훈련
compile()메서드 사용
에포트 5번
\\
그리고 활성화 함수에 알아보자
"""
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5667 - accuracy: 0.8072
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4064 - accuracy: 0.8542
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3711 - accuracy: 0.8666
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3473 - accuracy: 0.8745
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3308 - accuracy: 0.8803


In [12]:
"""
- 렐루 함수
- 렐루 함수는 심층 신경망에 좋다
렐루 함수는 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 
통과시키고 음수일 경우에는 0으로 만듬

렐수 함수는 특히 이미지 처리에 좋은 성능을 낸다

1차원으로 펼쳐주는 방법- 케라스에서는 Flatten 층이 있다
Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 함
Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라고 부름
"""
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [13]:
"""
첫 번쨰 Dense층에 있던 input_shape 매개변수를 Flatten 층으로 옮김
첫 번째 Dense층의 활성화 함수를 relu로 변경
summary()호출
"""
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [14]:
"""
훈련 데이터 다시 준비해서 모델에 훈련
"""
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
train_scaled, train_target, test_size=0.2, random_state=42)

In [15]:
"""
모델 컴파일하고 훈련

시그모디으 함수를 사용했을 때와 비교하면 성능이 조금 향상됨
"""
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5291 - accuracy: 0.8121
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3905 - accuracy: 0.8600
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3524 - accuracy: 0.8739
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3337 - accuracy: 0.8804
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3168 - accuracy: 0.8867


In [16]:
"""
검증 세트에서의 성능 확인
인공 신경망의 하이퍼파라미터에 알아보자
"""
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3574 - accuracy: 0.8809


[0.3573974668979645, 0.8809166550636292]

In [17]:
"""
-옵티마이저

신경망에는 특히 하이퍼파라미터가 많다 - 은닉층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개변수, 에포크 등등

마지막으로 compile()메서드에서는 케라스의 기본 경사 하강법 알고리즘인 RMSprop을 사용함
케라스는 다양한 종류의 경사 하강법 알고리즘을 제공함 - 이를 옵티마이저 

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD

SGD옵티마이저를 사용하려면 compile()메서드의 optimizer 매개변수를 "sgd"로 지정


model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics="accuracy")


만약 SGD클래스의 학습률 기본값이 0.01 일때 이를 바꾸고 싶다면 
원하는 학습률을 learning_rate 매개변수에 지정하여 사용

sgd = keras.optimizers.SGD(learning_rate = 0.1)

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공 
SGD 클래스의 momentum 매개변수의 기본값은 0
이를 0 보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도 처럼 사용하는 - 모멘텀 최적화를 사용함
보통 momentum 매개변수는 0.9 이상을 지정

SGD 클래스의 nesterov 매개변수를 기본값 false 에서 True 로 바꾸면 - 네스테로프 모멘텀 최적화 를 사용함
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현함
대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다 
이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높습니다 - 이런 학습률을 적응적 학습률 이라고 함

적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSProp 임
optimizer 매개변수의 기본값은 바로 "rmsprop"

두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용

adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss="sprase_categorical_crossentropy", metrics="accuracy")

RMSprop 모습
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizers=rmsprop, loss="sparse_categorical_crossentropy", metrics="accuracy")

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam

Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련

- 와 이렇게 중요한 옵티마이저 설명은 좋은데 한번 써먹으면 어떤지 
"""
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [18]:
"""
compile()메서드의 optimizer를 "adam"으로 설정하고 5번 에포크
"""
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5209 - accuracy: 0.8183
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3925 - accuracy: 0.8593
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3500 - accuracy: 0.8738
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3239 - accuracy: 0.8828
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3079 - accuracy: 0.8871


In [19]:
"""
검증 세트 확인
"""
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3440 - accuracy: 0.8750


[0.34397852420806885, 0.875]